# index
### 1. library import, data download
### 2. 전처리함수 적용
### 3. 파생변수생성 함수 적용
### 4. LGBM 모델 훈련
### 5. 교차검증 점수 확인
### 6. 제출파일 생성

# 1. library import, data download

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
import seaborn as sns
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour, mim)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = '/kaggle/input/dacon-lg/'

In [19]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_prob = pd.read_csv(PATH+"train_problem_data.csv")
test_err = pd.read_csv(PATH+'test_err_data.csv')
display(train_err)
display(test_err)

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0
16532644,44998,20201130211831,model_1,04.16.3553,31,1
16532645,44998,20201130211832,model_1,04.16.3553,15,1
16532646,44998,20201130212259,model_1,04.16.3553,16,1


# 2. preprocessing 함수 적용

결측값 제거

In [20]:
# 결측치를 대치할때 곧바로 fillna를 적용하기 위해서 inplace=True로 설정한다
# errcode의 최빈값은 0으로 확인되었기때문에 0으로 대치한다.
train_err.errcode.fillna('0', inplace=True)
display(train_err.isnull().sum())
test_err.errcode.fillna('0', inplace=True)
display(test_err.isnull().sum())

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     0
dtype: int64

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     0
dtype: int64

In [3]:
def preprocessing(data):
    # time의 최소값
    data_time_min = make_datetime(data.time.min())
    # time을 datetime형태로 변경
    data["datetime"] = data["time"].apply(make_datetime)
    
    # 경과일 컬럼 생성
    data["days"] = (data["datetime"] - data_time_min).dt.days + 1
    # 시간대 컬럼 생성
    data["hour"] = data["datetime"].dt.hour
    # 요일 컬럼 생성
    data["weekday"] = data["datetime"].dt.weekday
    
    del data["time"], data["datetime"]
    return data

In [21]:
start_minute = dt.datetime.now()

preprocessing(train_err)
preprocessing(test_err)

end_minute = dt.datetime.now()
print(f"경과시간: {(end_minute - start_minute)}")

display(train_err)
display(test_err)

경과시간: 0:01:11.410527


,user_id,model_nm,fwver,errtype,errcode,days,hour,weekday
0,10000,model_3,05.15.2138,15,1,1,2,6
1,10000,model_3,05.15.2138,12,1,1,3,6
2,10000,model_3,05.15.2138,11,1,1,3,6
3,10000,model_3,05.15.2138,16,1,1,5,6
4,10000,model_3,05.15.2138,4,0,1,5,6
...,...,...,...,...,...,...,...,...
16554658,24999,model_3,05.15.2138,15,1,30,16,0
16554659,24999,model_3,05.15.2138,16,1,30,17,0
16554660,24999,model_3,05.15.2138,4,0,30,17,0
16554661,24999,model_3,05.15.2138,4,0,30,17,0


,user_id,model_nm,fwver,errtype,errcode,days,hour,weekday
0,30000,model_1,04.16.3553,31,1,1,3,6
1,30000,model_1,04.16.3553,33,2,1,3,6
2,30000,model_1,04.16.3553,15,1,1,3,6
3,30000,model_1,04.16.3553,22,1,1,3,6
4,30000,model_1,04.16.3553,11,1,1,3,6
...,...,...,...,...,...,...,...,...
16532643,44998,model_1,04.16.3553,40,0,30,21,0
16532644,44998,model_1,04.16.3553,31,1,30,21,0
16532645,44998,model_1,04.16.3553,15,1,30,21,0
16532646,44998,model_1,04.16.3553,16,1,30,21,0


# 3. 파생변수생성 함수 적용 (make_cols)

In [22]:
# model_nm과 errtype은 train과 test가 동일함을 확인
# fwver과 errcode는 겹치는것과 겹치지않는 것이 있기때문에 차집합을 정리한다.
fwver_only_train = np.setdiff1d(train_err.fwver.unique(), test_err.fwver.unique())
fwver_only_test = np.setdiff1d(test_err.fwver.unique(), train_err.fwver.unique())

errcode_only_train = np.setdiff1d(train_err.errcode.unique(), test_err.errcode.unique())
errcode_only_test = np.setdiff1d(test_err.errcode.unique(), train_err.errcode.unique())

errcode_inter = np.intersect1d(train_err.errcode.unique(), test_err.errcode.unique())

print("train에만 있는 fwver 갯수:", len(fwver_only_train))
print("test에만 있는 fwver 갯수:", len(fwver_only_test))

print("train에만 있는 errcode 갯수:", len(errcode_only_train))
print("test에만 있는 errcode 갯수:", len(errcode_only_test))

print("train에만 있는 fwver:\n", fwver_only_train)
print("test에만 있는 fwver:\n", fwver_only_test)

print("train에만 있는 errcode:\n", errcode_only_train)
print("test에만 있는 errcode:\n", errcode_only_test)

print("train과 test의 errcode 교집합 갯수:", len(errcode_inter))
print("train과 test 의 교집합 errcode:\n", errcode_inter)

train에만 있는 fwver 갯수: 6
test에만 있는 fwver 갯수: 9
train에만 있는 errcode 갯수: 1398
test에만 있는 errcode 갯수: 1548
train에만 있는 fwver:
 ['04.16.2641' '04.16.3345' '04.22.1442' '04.33.1095' '05.15.2090'
 '05.15.2122']
test에만 있는 fwver:
 ['04.22.1170' '04.22.1448' '04.22.1478' '04.22.1608' '04.22.1772'
 '04.73.2569' '04.73.2577' '10.22.1770' '10.22.1780']
train에만 있는 errcode:
 ['10005' '10018' '10073' ... 'Y-00004' 'Y-00005' 'http']
test에만 있는 errcode:
 ['-1010' '10020' '10029' ... 'eDes' 'me="' 'tVer']
train과 test의 errcode 교집합 갯수: 1407
train과 test 의 교집합 errcode:
 ['-269' '-270' '0' ... 'scanning timeout' 'standby'
 'terminate by peer user']


In [ ]:
# data는 파생변수를 도출해내는 원천 데이터
# no_fwver는 data에 없는 유니크한 fwver들
# errcode_inter는 train과 test의 교집합 errcode

def make_cols(data, no_fwver, errcode_inter):
    print("initiate the function")
    #1) 인덱스가 user_id 수 만큼있는 df생성
    data_total = pd.DataFrame(index=data.user_id.unique())
    user_id_min = data.user_id.unique().min()
    user_id_max = data.user_id.unique().max()
    
    #######################################################################
    
    #2) model_nm 카운팅, 변화횟수 카운팅
    model_nm_dummies = pd.get_dummies(data["model_nm"])
    model_nm_dummies = pd.concat([data.user_id, model_nm_dummies], axis=1)

    # 각 아이디당 모델의 출현횟수
    model_nm_dummies = model_nm_dummies.groupby(model_nm_dummies.user_id).sum()

    # 하지만 우리가 알고 싶은 것은 각 id에 어떤 모델이 있냐 없냐만 알고싶기때문에 counting 한것을 모두 1로 변환
    # 즉, 해당 model_nm이 나타나면 1 아니면 0으로 바꾼다.
    def one_zero(num):
        if num != 0:
            num = 1
        return num

    for col in model_nm_dummies.columns:
        model_nm_dummies[str(col)] = model_nm_dummies[str(col)].apply(one_zero)

    # 그리고 한 id가 몇개의 모델을 갖고 있는지 확인하기위해
    # 한 행에서 출현한 숫자를 모두 더한다
    # model_change_cnt가
    # 1이면 1개의 모델
    # 2이면 2개의 모델 -> 1번 변화
    # 3이면 3개의 모델 -> 2번 변화
    change_cnt = []
    for i in range(len(model_nm_dummies.index)):
        change_cnt.append(model_nm_dummies.iloc[i].sum())

    model_nm_dummies["model_change_cnt"] = change_cnt

    #display(model_nm_dummies)
    # model_change_cnt의 value_counts를 시도하여 모델변화가 있는 사용자가 몇명인지 알아낼 수 있다.
    print(model_nm_dummies.model_change_cnt.value_counts())
    
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, model_nm_dummies], axis=1)
    # 메모리 절약
    del model_nm_dummies
    print("1) model_nm_dummies completed")
    ###############################################################
    
    #2) fwver 카운팅, 변화횟수 카운팅
    fwver_dummies = pd.get_dummies(data["fwver"])
    fwver_dummies = pd.concat([data.user_id, fwver_dummies], axis=1)

    # 각 아이디당 fwver의 출현횟수
    fwver_dummies = fwver_dummies.groupby(fwver_dummies.user_id).sum()

    # 하지만 우리가 알고 싶은 것은 각 id에 어떤 fwver이 있냐 없냐만 알고싶기때문에 counting 한것을 모두 1로 변환
    # 즉, 해당 Fwver이 나타나면 1 아니면 0으로 바꾼다.
    for col in fwver_dummies.columns:
        fwver_dummies[str(col)] = fwver_dummies[str(col)].apply(one_zero)
    
    # data에 없는 fwver을 원소가 0인 column으로 추가한다.
    for fw in no_fwver:
        fwver_dummies[fw] = np.zeros((len(data.user_id.unique())))
    # 그리고 한 id가 몇개의 fwver을 갖고 있는지 확인하기위해
    # 한 행에서 출현한 숫자를 모두 더한다
    # fwver_change_cnt가
    # 1이면 1개의 fwver
    # 2이면 2개의 fwver -> 1번 변화
    # 3이면 3개의 fwver -> 2번 변화
    # 4이면 4개의 fwver -> 3번 변화
    change_cnt = []
    for i in range(len(fwver_dummies.index)):
        change_cnt.append(fwver_dummies.iloc[i].sum())

    fwver_dummies["fwver_change_cnt"] = change_cnt

    #display(fwver_dummies)
    # fwver_change_cnt의 value_counts를 시도하여 fwver 변화가 있는 사용자가 몇명인지 알아낼 수 있다.
    print(fwver_dummies.fwver_change_cnt.value_counts())
    
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, fwver_dummies], axis=1)
    # 메모리 절약
    del fwver_dummies, change_cnt
    print("2) model_nm_dummies completed")
    ###############################################################
    
    #3) 모든 errtype이 발생한 days의 평균
    # 유니크한 user_id 만큼의 인덱스만 있는 dataframe 생성
    errtype_days_mean = pd.DataFrame(index=data.user_id.unique())
    # errtype의 유니크 값
    errtypes = np.sort(data.errtype.unique())

    for t in tqdm(errtypes):
        # 에러타입이 t인 row로 이뤄진 dataframe
        each_errtype = data.loc[data["errtype"] == t]
        # 그 dataframe(each_errtype)에서 유니크한 id를 추출
        id_uniq = set(each_errtype.user_id)
        errtype_days_ls = []
        for i in range(user_id_min, user_id_max+1):
            # id min ~ (max+1)에서 id_uniq에 id가 있으면 days의 평균을 구하고,
            if i in id_uniq:
                errtype_days_ls.append(each_errtype.loc[each_errtype["user_id"] == i].days.mean())
            # 없으면 0
            else:
                errtype_days_ls.append(0)
        # 마지막에 다 모아놓은 list를 한 컬럼으로 치고 이어붙이기
        errtype_days_mean["errtype"+str(t)+"days_mean"] = errtype_days_ls
        
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, errtype_days_mean], axis=1)
    # 메모리 절약
    del errtype_days_mean
    print("3) errtype_days_mean completed")
    ###############################################################
    
    #4) 모든 errtype이 발생한 hour의 평균
    # user_id 15000개의 인덱스만 있는 dataframe 생성
    errtype_hour_mean = pd.DataFrame(index=data.user_id.unique())
    # errtype의 유니크 값
    errtypes = np.sort(data.errtype.unique())

    for t in tqdm(errtypes):
        # 에러타입이 t인 row로 이뤄진 dataframe
        each_errtype = data.loc[data["errtype"] == t]
        # 그 dataframe(each_errtype)에서 유니크한 id를 추출
        id_uniq = set(each_errtype.user_id)
        errtype_hour_ls = []
        for i in range(user_id_min, user_id_max+1):
            # id user_id_min ~ user_id_max에서 id_uniq에 id가 있으면 hour의 평균을 구하고,
            if i in id_uniq:
                errtype_hour_ls.append(each_errtype.loc[each_errtype["user_id"] == i].hour.mean())
            # 없으면 0
            else:
                errtype_hour_ls.append(0)
        # 마지막에 다 모아놓은 list를 한 컬럼으로 치고 이어붙이기
        errtype_hour_mean["errtype"+str(t)+"_hour_mean"] = errtype_hour_ls
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, errtype_hour_mean], axis=1)
    # 메모리 절약
    del errtype_hour_mean
    print("4) errtype_hour_mean completed")
    ###############################################################
    
    #5) 모든 errtype이 발생한 weekday의 평균
    # user_id 15000개의 인덱스만 있는 dataframe 생성
    errtype_weekday_mean = pd.DataFrame(index=data.user_id.unique())
    # errtype의 유니크 값
    errtypes = np.sort(data.errtype.unique())

    for t in tqdm(errtypes):
        # 에러타입이 t인 row로 이뤄진 dataframe
        each_errtype = data.loc[data["errtype"] == t]
        # 그 dataframe(each_errtype)에서 유니크한 id를 추출
        id_uniq = set(each_errtype.user_id)
        errtype_weekday_ls = []
        for i in range(user_id_min, user_id_max+1):
            # id 10000 ~ 24999에서 id_uniq에 id가 있으면 weekday의 평균을 구하고,
            if i in id_uniq:
                errtype_weekday_ls.append(each_errtype.loc[each_errtype["user_id"] == i].weekday.mean())
            # 없으면 0
            else:
                errtype_weekday_ls.append(0)
        # 마지막에 다 모아놓은 list를 한 컬럼으로 치고 이어붙이기
        errtype_weekday_mean["errtype"+str(t)+"_weekday_mean"] = errtype_weekday_ls
        
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, errtype_weekday_mean], axis=1)
    # 메모리 절약
    del errtype_weekday_mean
    print("5) errtype_weekday_mean completed")
    ###############################################################
     
    # 6) errcode가 1인 row를 추출
    errcode_1_dummies = pd.get_dummies(data["errcode"] == "1")
    # data의 user_id를 index로 설정
    errcode_1_dummies = pd.concat([data.user_id, errcode_1_dummies.iloc[:,1]], axis=1)
    # user_id로 groupby.sum() 실행
    errcode_1_dummies = errcode_1_dummies.groupby(errcode_1_dummies.user_id).sum()
    # 컬럼명 변경
    errcode_1_dummies.columns = ["errcode_1"]
    #display(errcode_1_dummies)
    
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, errcode_1_dummies], axis=1)
    # 메모리 절약
    del errcode_1_dummies
    print("6) errcode_1_dummies completed")
    ###############################################################
    
    # 7) errcode가 0인 row를 추출
    errcode_0_dummies = pd.get_dummies(data["errcode"] == "0")
    # data의 user_id를 index로 설정
    errcode_0_dummies = pd.concat([data.user_id, errcode_0_dummies.iloc[:,1]], axis=1)
    # user_id로 groupby.sum() 실행
    errcode_0_dummies = errcode_0_dummies.groupby(errcode_0_dummies.user_id).sum()
    # 컬럼명 변경
    errcode_0_dummies.columns = ["errcode_0"]
    #display(errcode_0_dummies)
    
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, errcode_0_dummies], axis=1)
    # 메모리 절약
    del errcode_0_dummies
    print("7) errcode_0_dummies completed")
    ###############################################################
    
    # 8) train과 test의 교집합 errcode에서 문자형태의 코드를 하나의 컬럼으로 생성
    # - 문자는 문자그대로, 정수는 정수형태로 출력하는 함수
    def int2bInt(x):
        # 문자로 구성된 errcode를 찾는다
        searched = re.search('[a-zA-Z]+', str(x))
        # 0으로 시작하지만 모두 숫자인 errcode를 찾는다 (ex. 0001)
        matched = re.match('0{2,}', str(x))
        if searched:
            result = x
        elif matched:
            result = x
        else:
            result = int(float(x)) # 위 조건이 아닌 errcode(정수인 errcode)는 int로 바꾼다.
        return result

    int_errcode = [] # 정수는 정수로, 문자는 str로 담겨져있는 list

    for code in errcode_inter:
        int_errcode.append(int2bInt(code))
        
    word_err = []
    for err in int_errcode:
        if type(err) == str:
            word_err.append(err)
    # 문자형태의 errcode 컬럼 생성 시작            
    word_vari = pd.DataFrame(index=data.user_id.unique())

    for code in tqdm(word_err):
        # data.user_id의 컬럼만 갖고있는 word_frame DataFrame 생성
        word_frame = pd.DataFrame(data.user_id)
        # errcode가 code인 row를 추출
        word_dummies = pd.get_dummies(data["errcode"] == code)
        # 컬럼명을 code로 하는 column 생성
        word_frame[code] = word_dummies.iloc[:,1] #두번째 컬럼이 조건식 True인 column
        # user_id로 groupby.sum() 실행
        word_frame = word_frame.groupby(word_frame.user_id).sum()
        # 최종 variables dataframe에 그룹바이 한 column 추가
        word_vari[code] = word_frame[code]
        
    # 컬럼 이어붙이기
    data_total = pd.concat([data_total, word_vari], axis=1)
    # 메모리 절약
    del word_vari
    print("8) word_vari completed")
    ################################################################
   
    
    return data_total

test_err 에서 user_id 43262가 없음!!!!

In [54]:
#train_cols = make_cols(train_err, fwver_only_test, errcode_inter)
#test_cols = make_cols(test_err, fwver_only_train, errcode_inter)

#train_cols = pd.read_csv("/kaggle/input/errtype-days-mean/train_cols.csv")
test_cols = pd.read_csv("/kaggle/input/errtype-days-mean/test_cols.csv")

display(train_cols)
display(test_cols)

,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_change_cnt,...,V-21008,Y-00008,active,connection fail for LMP response timout,connection fail to establish,connection timeout,connectionterminated by local host,scanning timeout,standby,terminate by peer user
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,126.0,0,0.0,0.0,0,0,625.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,12.0,0,0.0,7.0,0,0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,7.0,0,1.0,104.0,0,0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,4.0,0,0.0,0.0,0,0,8.0,0.0
14996,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,0.0,0,0.0,0.0,0,0,0.0,0.0
14997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,4.0,0,0.0,13.0,1,0,11.0,0.0
14998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,1.0,0,0.0,0.0,0,0,0.0,0.0


,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_change_cnt,...,V-21008,Y-00008,active,connection fail for LMP response timout,connection fail to establish,connection timeout,connectionterminated by local host,scanning timeout,standby,terminate by peer user
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0,0,141.0,0.0,55.0,1107.0,0.0,0,4.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,7.0,0.0,0.0,26.0,0.0,0,6.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,1.0,0.0,0.0,1.0,0.0,0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0,0,5.0,0.0,0.0,0.0,0.0,0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0.0,0.0,7.0,197.0,0.0,0,0.0,0.0
14994,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
14995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,104.0,0.0,4.0,44.0,0.0,0,11.0,2.0
14996,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,1655.0,0.0,1010.0,10449.0,1.0,3,20.0,1.0


In [59]:
new_data = pd.DataFrame(columns=test_cols.columns)

In [61]:
new_data.loc[0] = [0 for x in range(252)]
display(new_data)

,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_change_cnt,...,V-21008,Y-00008,active,connection fail for LMP response timout,connection fail to establish,connection timeout,connectionterminated by local host,scanning timeout,standby,terminate by peer user
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
new_data = pd.DataFrame(columns=test_cols.columns)
new_data.loc[0] = [0 for x in range(252)] # 0으로 구성된 빈 데이터
idx = 13262 # 빠진 인덱스

temp1 = test_cols[test_cols.index < idx]
temp2 = test_cols[test_cols.index >= idx]

test_cols1 = temp1.append(new_data,ignore_index=True).append(temp2, ignore_index=True)

In [63]:
test_cols1

,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_change_cnt,...,V-21008,Y-00008,active,connection fail for LMP response timout,connection fail to establish,connection timeout,connectionterminated by local host,scanning timeout,standby,terminate by peer user
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0,0,141.0,0.0,55.0,1107.0,0.0,0,4.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,7.0,0.0,0.0,26.0,0.0,0,6.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,1.0,0.0,0.0,1.0,0.0,0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0,0,5.0,0.0,0.0,0.0,0.0,0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0.0,0.0,7.0,197.0,0.0,0,0.0,0.0
14995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
14996,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,104.0,0.0,4.0,44.0,0.0,0,11.0,2.0
14997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,1655.0,0.0,1010.0,10449.0,1.0,3,20.0,1.0


# 4. modeling

problem data

In [6]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [7]:
# 변수 이름 변경
# train_cols  -> train_x
# problem-> train_y

train_x = np.array(train_cols)
train_y = problem
#del error, problem
print(train_x.shape)
print(train_y.shape)

(15000, 252)
(15000,)


In [11]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015819 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23284
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 184
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[9]	valid_0's auc: 0.81987	valid_0's pr_auc: 0.823549
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015264 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23548
[LightGBM] [Info] Number of data points in the train set: 12000, number of used featu

# 4. 교차검증 점수 확인

In [12]:
print(np.mean(auc_scores))

0.8068467964741359


# 5. 제출파일 생성

In [64]:
test_x = np.array(test_cols1)

In [65]:
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [66]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [67]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [68]:
sample_submssion.to_csv("dacon_baseline.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.597682
1,30001,0.338777
2,30002,0.459386
3,30003,0.566163
4,30004,0.615405
...,...,...
14994,44994,0.322272
14995,44995,0.294762
14996,44996,0.479270
14997,44997,0.441644
